# Recolección de datos

Hecha la investigación para identificar fuentes confiables de datos, vamos a proceder a descargar los valores históricos de los activos e índices deseados, para posteriormente utilizarlos en la construcción de modelos.

Importamos las dependencias necesarias:

In [24]:
import requests
import pandas as pd
import yfinance as yf
from pathlib import Path
from pytrends.request import TrendReq
from datetime import datetime, timedelta

from dotenv import load_dotenv
import os

# Cargar variables desde el archivo .env
load_dotenv()

True

Lo primero es obtener los datos históricos diarios del precio de bitcoin, disponibles en yahoo finance:

In [25]:
# Descargar histórico completo de BTC/USD desde 2009 (o el mayor histórico disponible)
btc = yf.download("BTC-USD", start="2009-01-01")

/var/folders/c_/rq9m5qb17jsg0xbdp8j28n2c0000gn/T/ipykernel_5861/947041190.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc = yf.download("BTC-USD", start="2009-01-01")
[*********************100%***********************]  1 of 1 completed


Revisamos la primera fecha para la que tenemos disponible el precio, de tal forma de utilizar como fecha base para obtener el resto de activos o índices a utilizar, además de conocer de forma temprana el período de tiempo para el que podremos trabajar:

In [26]:
start_date = btc.index.min()
print("Primera fecha disponible:", start_date.date())

Primera fecha disponible: 2014-09-17


In [27]:
# Crear carpeta si no existe
output_dir = "../datasets/btc"
os.makedirs(output_dir, exist_ok=True)

# Agrupar por año
for year in btc.index.year.unique():
    btc_year = btc[btc.index.year == year]
    output_path = os.path.join(output_dir, f"btc_{year}.csv")
    btc_year.to_csv(output_path)

Aprovechamos la biblioteca de yahoo finance para obtener valores de otros activos: Ethereum, Oro y los índices S&P 500 y DXY (dollar index):

In [28]:
# Lista de activos: (activo, símbolo_yfinance)
assets = [
    ("eth", "ETH-USD"),
    ("gold", "GC=F"),
    ("sp500", "^GSPC"),
    ("dxy", "DX-Y.NYB")
]

for name, symbol in assets:
    print(f"⬇ Descargando datos de {name.upper()} ({symbol})...")
    data = yf.download(symbol, start=start_date.date(), auto_adjust=True)

    # Crear carpeta de salida: datasets/{activo}
    output_dir = Path("../datasets") / name
    output_dir.mkdir(parents=True, exist_ok=True)

    # Agrupar por año
    for year in data.index.year.unique():
        data_year = data[data.index.year == year]
        output_path = os.path.join(output_dir, f"{name}_{year}.csv")
        data_year.to_csv(output_path)

    print(f"✔ Datos de {name.upper()} guardados en: {output_dir.resolve()}\n")

⬇ Descargando datos de ETH (ETH-USD)...


[*********************100%***********************]  1 of 1 completed


✔ Datos de ETH guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/eth

⬇ Descargando datos de GOLD (GC=F)...


[*********************100%***********************]  1 of 1 completed


✔ Datos de GOLD guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/gold

⬇ Descargando datos de SP500 (^GSPC)...


[*********************100%***********************]  1 of 1 completed


✔ Datos de SP500 guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/sp500

⬇ Descargando datos de DXY (DX-Y.NYB)...


[*********************100%***********************]  1 of 1 completed

✔ Datos de DXY guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/dxy



De Google Trends descargamos los datos relativos a las búsquedas de bitcoin a lo largo del tiempo.
Google Trends maneja valores entre 0 y 100.

In [29]:
import time

# --- Parámetros de Google Trends ------------------------------------
kw_list   = ["bitcoin"]
start_date_gt = start_date.to_pydatetime()
end_date_gt   = datetime(2025, 6, 15)
delta      = timedelta(days=270)            # rango máximo permitido
pytrends   = TrendReq(hl="en-US", tz=0)

# --- Directorio de salida -------------------------------------------
output_dir = "../datasets/trend"
os.makedirs(output_dir, exist_ok=True)

# --- Descarga por tramos y guarda por año ---------------------------
current_start = start_date_gt

while current_start < end_date_gt:
    # El rango superior es inclusivo, por eso restamos un día cuando superamos end_date
    current_end = min(current_start + delta, end_date_gt)
    timeframe   = f"{current_start:%Y-%m-%d} {current_end:%Y-%m-%d}"
    print(timeframe)

    try:
        pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo="", gprop="")
        data = pytrends.interest_over_time()

        # Salta rangos sin datos
        if data.empty:
            print(f"No hay datos para: {timeframe}")
        else:
            # Limpieza: quita la columna 'isPartial' si existe
            data = data.drop(columns="isPartial", errors="ignore")

            #  Agrupar por año y escribir/añadir CSV
            for year, year_df in data.groupby(data.index.year):
                filename      = f"{output_dir}/bitcoin_trend_{year}.csv"
                write_header  = not os.path.isfile(filename) # 1º vez → incluye encabezado
                year_df.to_csv(filename, mode="a", header=write_header)
                print(f"✓ Guardado/actualizado: {filename}")

    except Exception as e:
        print(f"Error al procesar {timeframe}: {e}")

    # Avanza un día después del último día descargado para evitar solaparse
    current_start = current_end + timedelta(days=1)
    time.sleep(5)

2014-09-17 2015-06-14
Error al procesar 2014-09-17 2015-06-14: The request failed: Google returned a response with code 429
2015-06-15 2016-03-11
Error al procesar 2015-06-15 2016-03-11: The request failed: Google returned a response with code 429
2016-03-12 2016-12-07
Error al procesar 2016-03-12 2016-12-07: The request failed: Google returned a response with code 429
2016-12-08 2017-09-04
Error al procesar 2016-12-08 2017-09-04: The request failed: Google returned a response with code 429
2017-09-05 2018-06-02
Error al procesar 2017-09-05 2018-06-02: The request failed: Google returned a response with code 429
2018-06-03 2019-02-28
Error al procesar 2018-06-03 2019-02-28: The request failed: Google returned a response with code 429
2019-03-01 2019-11-26
✓ Guardado/actualizado: ../datasets/trend/bitcoin_trend_2019.csv
2019-11-27 2020-08-23
Error al procesar 2019-11-27 2020-08-23: The request failed: Google returned a response with code 429
2020-08-24 2021-05-21
✓ Guardado/actualizado:

/Users/cbarril/dev/posgrado/tp_ast1_19co2024/ast1/lib/python3.12/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


✓ Guardado/actualizado: ../datasets/trend/bitcoin_trend_2025.csv


Tuvimos algunos problemas con la API de Google Trends, pero todos los datos necesarios fueron descargados luego de algunos intentos.

Obtenemos de la API de Alternative los datos del índice de miedo y codicia:

In [34]:
# --- Directorio de salida -------------------------------------------
output_dir = "../datasets/fear_and_greed_index"
os.makedirs(output_dir, exist_ok=True)

url = "https://api.alternative.me/fng/?limit=0&format=json"
response = requests.get(url)
data = response.json()["data"]

# Convertir a DataFrame
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
df['value'] = df['value'].astype(int)
df = df.sort_values('timestamp')

# Ver columnas disponibles
print(df.head())

# Guardar
filename = f"{output_dir}/fear_and_greed_index.csv"
df.to_csv(filename, index=False)

print(f"✔ Datos de fear and greed index guardados en: {filename}\n")

      value value_classification  timestamp time_until_update
2687     30                 Fear 2018-02-01               NaN
2686     15         Extreme Fear 2018-02-02               NaN
2685     40                 Fear 2018-02-03               NaN
2684     24         Extreme Fear 2018-02-04               NaN
2683     11         Extreme Fear 2018-02-05               NaN
✔ Datos de fear and greed index guardados en: ../datasets/fear_and_greed_index/fear_and_greed_index.csv



Importante identificar que solo contamos con datos desde el 2018. Veremos luego si es útil este índice o debemos prescindir de él.

Obtenemos otra información que suele relacionarse a tendencias en bitcoin, que es la cantidad de direcciones (billeteras) activas.

In [31]:
# --- Descargar datos de direcciones activas ---------------------
url = "https://api.blockchain.info/charts/n-unique-addresses?timespan=all&format=csv"
df = pd.read_csv(url)
df.columns = ['date', 'active_addresses']
df['date'] = pd.to_datetime(df['date'])

# --- Filtrar desde start_date -----------------------------------
df = df[df['date'] >= start_date]

# --- Crear carpeta destino --------------------------------------
base_path = "../datasets/active_addresses"
os.makedirs(base_path, exist_ok=True)

# --- Separar y guardar por año ----------------------------------
for year, group in df.groupby(df['date'].dt.year):
    file_path = os.path.join(base_path, f"active_addresses_{year}.csv")
    group.to_csv(file_path, index=False)
    print(f"✓ Guardado: {file_path}")

✓ Guardado: ../datasets/active_addresses/active_addresses_2014.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2015.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2016.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2017.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2018.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2019.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2020.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2021.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2022.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2023.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2024.csv
✓ Guardado: ../datasets/active_addresses/active_addresses_2025.csv


Finalmente, de la API de la FED de St Louis, previo gestión de una API_KEY, obtenemos los datos históricos de la tasa de interés:

In [32]:
# --- Clave de API y parámetros para la FED ----------------------
fed_api_key = os.getenv("FED_API_KEY")
series_id = "FEDFUNDS"
url = "https://api.stlouisfed.org/fred/series/observations"
params = {
    "series_id": series_id,
    "api_key": fed_api_key,
    "file_type": "json",
}

# --- Llamado a la API -------------------------------------------
response = requests.get(url, params=params)
data = response.json()["observations"]

# --- Crear DataFrame --------------------------------------------
df = pd.DataFrame(data)[["date", "value"]]
df.columns = ["date", "interest_rate"]
df["date"] = pd.to_datetime(df["date"])
df["interest_rate"] = pd.to_numeric(df["interest_rate"], errors="coerce")

# --- Filtrar desde la fecha de inicio --------------------------
df = df[df["date"] >= start_date]

# --- Crear carpeta y guardar archivos por año ------------------
output_dir = "../datasets/interest_rate"
os.makedirs(output_dir, exist_ok=True)

for year, group in df.groupby(df["date"].dt.year):
    file_path = os.path.join(output_dir, f"interest_rate_{year}.csv")
    group.to_csv(file_path, index=False)
    print(f"✓ Guardado: {file_path}")


✓ Guardado: ../datasets/interest_rate/interest_rate_2014.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2015.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2016.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2017.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2018.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2019.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2020.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2021.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2022.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2023.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2024.csv
✓ Guardado: ../datasets/interest_rate/interest_rate_2025.csv
